In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use('Agg')

# import package
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from pyimagesearch.nn.conv import MiniVGGNet
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
def step_decay(epoch):
    # init the base learning rate, drop factor and epoch to drop
    initAlpha = 0.01
    factor = 0.5
    dropEvery = 5

    # compute learning rate for the current epoch
    alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))

    return float(alpha)

In [3]:
args = {
    'output' : 'output/lr_decay_f0.5_plot.png'
}

print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype('float') / 255.0
testX = testX.astype('float') / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# initialzie the label names
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

[INFO] loading CIFAR-10 data...


In [4]:
# define the set of callbacks to be passed to the modesl during training
callbacks = [LearningRateScheduler(step_decay)]

# init the optimizer and model
opt = SGD(lr=0.01, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train the network
H = model.fit(trainX, trainY, validation_data=(testX, testY), 
              batch_size=64, epochs=40, callbacks=callbacks, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 18s 360us/sample - loss: 1.6751 - accuracy: 0.4472 - val_loss: 1.2250 - val_accuracy: 0.5742
Epoch 2/40
50000/50000 [==============================] - 15s 299us/sample - loss: 1.1807 - accuracy: 0.5962 - val_loss: 1.0848 - val_accuracy: 0.6231
Epoch 3/40
50000/50000 [==============================] - 15s 299us/sample - loss: 0.9837 - accuracy: 0.6600 - val_loss: 0.8537 - val_accuracy: 0.7030
Epoch 4/40
50000/50000 [==============================] - 15s 299us/sample - loss: 0.8711 - accuracy: 0.6966 - val_loss: 0.8150 - val_accuracy: 0.7214
Epoch 5/40
50000/50000 [==============================] - 15s 299us/sample - loss: 0.7414 - accuracy: 0.7404 - val_loss: 0.6942 - val_accuracy: 0.7551
Epoch 6/40
50000/50000 [==============================] - 15s 299us/sample - loss: 0.6826 - accuracy: 0.7601 - val_loss: 0.6877 - val_accuracy: 0.7603
Epoch 7/40
50000/50000 [====================

In [5]:
# evalute the network
print("[info] evaluating the network..")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

[info] evaluating the network..
              precision    recall  f1-score   support

    airplane       0.86      0.85      0.85      1000
  automobile       0.93      0.91      0.92      1000
        bird       0.77      0.70      0.73      1000
         cat       0.68      0.64      0.66      1000
        deer       0.79      0.83      0.81      1000
         dog       0.73      0.75      0.74      1000
        frog       0.82      0.90      0.86      1000
       horse       0.87      0.85      0.86      1000
        ship       0.90      0.91      0.91      1000
       truck       0.88      0.90      0.89      1000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



In [6]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 40), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 40), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 40), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 40), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])